In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import cv2
import numpy as np
import gc

In [ ]:
!pip install fvcore
!pip install tqdm
!pip install pytorchvideo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
# Define the zip file path and the destination folder
zip_file_path = '/content/drive/MyDrive/Face_Frames2_Test.zip'
destination_folder = '/content/'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print("Unzipping completed.")

Unzipping completed.


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
def create_datasets(root_dir):
    datasets_list = []
    for i, trial_name in enumerate(os.listdir(root_dir)):
        trial_path = os.path.join(root_dir, trial_name)
        if os.path.isdir(trial_path):
            transform = transforms.Compose([
                transforms.Resize((224, 224)),  # Resize images to a standard size
                transforms.ToTensor()  # Convert images to PyTorch tensors
            ])
            dataset = datasets.ImageFolder(root=trial_path, transform=transform)
            datasets_list.append(dataset)
    return datasets_list

# Create datasets for each trial directory
deceptive_datasets = create_datasets('Face_Frames2_Test/Deceptive')
truthful_datasets = create_datasets('Face_Frames2_Test/Truthful')

In [ ]:
deceptive_dict = {}
for index, deceptive in enumerate(deceptive_datasets):
    # Assuming deceptive is your dataset
    deceptive_length = len(deceptive)
    num_samples = 30

    # List to hold the new dataset
    deceptive_data = []

    for i in range(0, deceptive_length, num_samples):
        # Get the current sequence of 30 samples
        combined_samples = []
        for j in range(num_samples):
            if i + j < deceptive_length:
                data, _ = deceptive[i + j]  # Extract the data part (ignore the label if present)
                combined_samples.append(data)


        # Stack the current sequence of samples along a new dimension
        if len(combined_samples) == num_samples:  # Ensure we have exactly 30 samples
            combined_samples = [combined_samples[0]] + combined_samples + [combined_samples[-1]]
            combined_tensor = torch.stack(combined_samples, dim=0)
            deceptive_data.append(combined_tensor.permute(1, 0, 2, 3))
    deceptive_dict[index] = deceptive_data

truthful_dict = {}
for index, truthful in enumerate(truthful_datasets):
    # Assuming deceptive is your dataset
    truthful_length = len(truthful)
    num_samples = 30

    # List to hold the new dataset
    truthful_data = []

    for i in range(0, truthful_length, num_samples):
        # Get the current sequence of 30 samples
        combined_samples = []
        for j in range(num_samples):
            if i + j < truthful_length:
                data, _ = truthful[i + j]  # Extract the data part (ignore the label if present)
                combined_samples.append(data)


        # Stack the current sequence of samples along a new dimension
        if len(combined_samples) == num_samples:  # Ensure we have exactly 30 samples
            combined_samples = [combined_samples[0]] + combined_samples + [combined_samples[-1]]
            combined_tensor = torch.stack(combined_samples, dim=0)
            truthful_data.append(combined_tensor.permute(1, 0, 2, 3))
    truthful_dict[index] = truthful_data


Each element of dict represent a video

Each video is represented by a list

Each element of the list is a tensor of 30 frames which is 1.5 subvideos



In [ ]:
len(deceptive_dict[7])
# Each element of dict represent a video
# Each video is represented by a list
# Each element of the a list is a tensor of 30 frames


19

In [ ]:
model = torch.load('/content/drive/MyDrive/Models/Adam_lr00001Schedular-WDe-2.pth', map_location=torch.device('cpu'))


In [ ]:
device = 'cuda'
logit_means = {}
final_predictions = {}

# Clear GPU memory before starting the loop
torch.cuda.empty_cache()
gc.collect()

model = model.to(device)
model.eval()

with torch.no_grad():
  for i in deceptive_dict:
      logits = []
      for tensor in deceptive_dict[i]:
          tensor = tensor.unsqueeze(0)
          logit = model(tensor.to(device))  # Modelden logit çıktısını al
          logits.append(logit)

      # Logitlerin ortalamasını al
      logit_mean = torch.mean(torch.stack(logits), dim=0)
      logit_means[i] = logit_mean  # Ortalamayı dictionary'e ekle

      final_prediction = torch.argmax(logit_mean, dim=1).cpu().numpy()
      final_predictions[i] = final_prediction  # Store the final prediction as a numpy array

deceptive_predictions = np.array(list(final_predictions.values()))
deceptive_true = np.ones(len(deceptive_predictions))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


In [ ]:
device = 'cuda'
logit_means = {}
final_predictions = {}

# Clear GPU memory before starting the loop
torch.cuda.empty_cache()
gc.collect()

model = model.to(device)
model.eval()

with torch.no_grad():
  for i in truthful_dict:
      logits = []
      for tensor in truthful_dict[i]:
          tensor = tensor.unsqueeze(0)
          logit = model(tensor.to(device))  # Modelden logit çıktısını al
          logits.append(logit)

      # Logitlerin ortalamasını al
      logit_mean = torch.mean(torch.stack(logits), dim=0)
      logit_means[i] = logit_mean  # Ortalamayı dictionary'e ekle

      final_prediction = torch.argmax(logit_mean, dim=1).cpu().numpy()
      final_predictions[i] = final_prediction  # Store the final prediction as a numpy array

truthful_predictions = np.array(list(final_predictions.values()))
truthful_true = np.zeros(len(deceptive_predictions))

In [ ]:
predictions = np.concatenate((deceptive_predictions, truthful_predictions))
true = np.concatenate((deceptive_true, truthful_true))


In [ ]:
accuracy = accuracy_score(true, predictions)
f1 = f1_score(true, predictions)
precision = precision_score(true, predictions)
recall = recall_score(true, predictions)
df = pd.DataFrame({'Accuracy': [accuracy], 'F1 Score': [f1], 'Precision': [precision], 'Recall': [recall]})

print(df)
df.to_csv('test_Adam_lr00001Schedular-WDe-2.csv', index=False)

   Accuracy  F1 Score  Precision    Recall
0  0.818182  0.818182   0.818182  0.818182
